<a href="https://colab.research.google.com/github/Alexrosulek/Cs50/blob/main/makearticle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  29253      0 --:--:-- --:--:-- --:--:-- 29291
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:4 http://security.ubuntu.com/ubun

In [6]:
!nohup ollama serve &
!ollama pull deepseek-r1:14b

!ollama pull qwen2.5:7b
!ollama pull qwen2.5:3b
!pip install ollama
#!unzip -q /content/processed_shops.zip

nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 G

In [ ]:
from pickle import FALSE
!nohup ollama serve &
import asyncio
import re
import json
import os
import subprocess
import hashlib
from urllib.parse import urlparse
from pathlib import Path
import ollama
import httpx
import requests
import asyncio
########################################################################
# Configuration
########################################################################
MODEL_SEMAPHORE = asyncio.Semaphore(1)  # Ensures only one model runs at a time
CHECKPOINT_FILE = "progress.checkpoint2"
BASE_INPUT_DIR = "processed_shops"      # Root directory containing input shop data
BASE_OUTPUT_DIR = "generated_articles"  # Root directory for generated articles
                   # Target states to process
MIN_CONTENT_LENGTH = 1000                # Minimum combined content length to process
CHUNK_SIZE = 10000
                 # Maximum text chunk size for summarization
SHOPDD = False
########################################################################
# AI Client Classes
########################################################################


class QwenClient:
    async def fast_summarize(self, text):
        """Use Qwen (0.5b) for business/SEO-focused summarization."""
        async with MODEL_SEMAPHORE:
            prompt = (
                "Your task is to extract key service details from this text while removing code and unwanted info\n"


                "STRICTLY IGNORE:\n"
                "- Business name/address (already known do not worry about)\n"

                "- Marketing fluff ('best in town')\n"
                "- Customer reviews\n"
                "- Repeated information\n\n"
                "FORMAT REQUIREMENTS:\n"
                "- Bullet points only\n"
                "No extra spaceing\n"
                "- No complete sentences\n"
                "- Preserve exact numbers/special & unique info\n\n"
                "Keep all concrete facts.\n"
                "No one is talking to you, you are extracting ALL info.\n"
              "REMOVE COOKIES AND PRIVACY POLICYS FULLY, "
              "REMOVE JAVASCRIPT, REMOVE RANDOM SCRIPTS, REMOVE COOKIES"
                "If error or empty or unsure don't respond. "
                    "REMOVE RANDOM WEBPAGE OR HTML OR CODE INFO, REMOVE CODE, KEEP SHOP INFO. "
                f"RAW INPUT TEXT:\n{text}"
            )

            try:

                response = ollama.chat(
                        model='qwen2.5:3b',
                        messages=[{'role': 'user', 'content': prompt}]
                    )

                response = response['message']['content'].strip()


                # Debug line for you—feel free to remove if you want fewer logs:
                print(f"[AI] QwenClient.fast_summarize response: {response}")

                # Fallback if response is empty
                if not response:
                    print("Empty summary response, returning partial original text.")
                    return text[: CHUNK_SIZE // 4]  # integer slice

                return response

            except Exception as e:
                print(f"Qwen error: {e}")
                # Fallback return if something goes wrong
                SHOPDD = True
                return text[: CHUNK_SIZE // 4]  # integer slice


async def chunked_summarize(qwen_client, text, chunk_size=CHUNK_SIZE):
    """
    Splits 'text' into chunk_size blocks, summarizes each,
    then combines those summaries into one final summary.
    This produces a single, concise summary without repeated partials.
    """

    # If it's already small enough, just return it.
    if len(text) <= chunk_size:

        summary = await qwen_client.fast_summarize(text)
        print(f"Text fits in one chunk (length={len(text)}). No further chunking needed.")
        return summary

    split_size = chunk_size // 2
    chunks = [text[i : i + split_size] for i in range(0, len(text), split_size)]
    print(f"Split into {len(chunks)} chunks ~{split_size} chars each.")

    # Step B: Summarize each chunk individually
    summarized_chunks = []
    for idx, chunk in enumerate(chunks, start=1):
        print("starting", idx)
        summary = await qwen_client.fast_summarize(chunk)
        if not summary:
            print("nosum")
            # Fallback if the model returns empty
            summary = chunk[: split_size // 4]
        summarized_chunks.append(summary)

    # Step C: Join all chunk summaries
    combined_summary = "\n".join(summarized_chunks)

    summary = await qwen_client.fast_summarize(combined_summary)

    return summary

class DeepSeekClient:


    async def generate_article(self, prompt):
        """Use DeepSeek 7b for article generation."""
        async with MODEL_SEMAPHORE:
            try:
                response = ollama.chat(
                        model='deepseek-r1:14b',
                        messages=[{'role': 'user', 'content': prompt}]
                    )

                response = response['message']['content'].strip()

                clean_response = self._clean_output(response)
                print(f"[AI] DeepSeekClient.generate_article response: {clean_response}")
                return clean_response
            except Exception as e:
                print(f"DeepSeek Article error: {e}")
                return ""



    async def validate_article(self, article):
        """Validation logic remains same"""
        async with MODEL_SEMAPHORE:
            prompt = (
                "You are an article checker. Please check the following article for these requirements. The article must be paragraphs, "
 "YOU ARE WORK FOR SELLPLASMANEAR.COM CHECKING ARTICLES WRITTEN ABOUT WHAT PLASMA DONATION CENTERS HAVE TO OFFER OFF GIVEN WEBSITE INFO WE GATHERED ON THEM FOR THE MAIN PAGE OF SELLPLASMANEAR.COM!\n"
                "MAKE SURE IT IS FRIENDY CONSUMER LANGUAGE READABLE ARTICLE ABOUT THE SPECIFIC SHOP SERVICES"
                "IF ITS ABOUT SOME HOLIDAY OR DEAL OR ISNT WHAT A READER WOULD EXPECT ON A REPORT ABOUT A SHOPS SERVICES SAY 'BAD'"
                "Article must be a article about a shop not anything else or somthing random."
                  "ENSURE THAT IT IS A PROPER ARTICLE BASED ON A SHOP. "
                 "DENY ARTICLES REFERING TO PRIVACY OR COOKIY POLICY, OR TOOL TO TRACK YOU, IT CANNOT TALK ABOUT THE WEBSITE CODE IT CAME FROM OR REFER TO IT ONLY THE SHOP ARTICLE"
                  "DENY ARTICLES WITH BAD FORMATTING OR HTML. "

                "INVALIDATE COOKIE AND PRIVACY POLICY ARTICLES"
                "must be aligned with writing from sellplasmanear.com about this shops details. "
                "NO markdown formatting, no *, extra commentary, or additional text beyond the core article content. NO EXTRA COMMENTS BEFORE OR AFTER THE ARTICLE, JUST THE ARTICLE. "
                "article must not mention seo tactics or anything bad or anything caused by its ai generation which is noticable and bad for a reader, must be consumer readable article."
                "must be based on a shop"
                "LONG IS GOOD"
                "Allow for faq "
                  "FORBIDDEN PHRASES:\n"
    "- 'Here is the article'\n"
    "- 'As a business'\n"
    "- 'In conclusion'\n"
    "- 'We recommend'\n\n"

    "- No markdown, bullets, or special characters\n"

    "\n No ###, no **, DO NOT USE THEM!"
    "NO HASHTAGS OR ASTERISKS"

    "- 'Deals"
    "- 'deal saturday'\n\n"
 "ARTICLE MUST BE FORMATTED IN <p style='line-height: 1.6;'>   </P> WITH INLINE CSS NOTHING EXTRA BEFORE OR AFTER <P style=''> </P>."

                "ONLY If the article meets these criteria, you MUST say 'VALID', IF GOOD SAY exactly 'VALID' ELSE SAY 'BAD'.  "
                "\n\n"
                f"Article:\n{article}\n"
            )
            try:
                response = ollama.chat(
                    model='qwen2.5:7b',
                    messages=[{'role': 'user', 'content': prompt}]
                )
                clean_response = self._clean_output(response['message']['content'].strip())
                return "valid" in clean_response.lower(), clean_response
            except Exception as e:
                print(f"Validation error: {e}")
                return False, "validation error"

    async def fixer(self, article, validation_response):
        """Targeted article repair based on validation feedback"""
        async with MODEL_SEMAPHORE:
            prompt = (
                "ARTICLE REPAIR PROTOCOL\n\n"
        "KEEP FAQS AND FULL LENGTH OF THE ARTICLE. "
 "YOU WORK FOR SELLPLASMANEAR.COM WRITING ABOUT WHAT SHOPS HAVE TO OFFER OFF GIVEN WEBSITE INFO WE GATHERED ON THEM FOR THE MAIN PAGE OF SELLPLASMANEAR.COM!\n"
                "MAKE SURE IT IS FRIENDY CONSUMER LANGUAGE READABLE ARTICLE ABOUT THE SPECIFIC SHOP SERVICES"
                "FIX THESE SPECIFIC ISSUES:\n"
                f"{validation_response}\n\n"
                "REPAIR RULES:\n"
                "MAKE SURE IT IS CONSUMER LANGUAGE READABLE ARTICLE"
                " MOST IMPORTANT! Remove ALL text before/after article content, RETURN NOTHING BUT THE ARTICLE\n"
                "ENSURE THAT IT IS A PROPER ARTICLE BASED ON A SHOP"
"Remove references not about the shop\n"
"remove cookie or privacy policy or code related things\n"
                " Preserve all business details\n"
                " Use natural consumer language\n\n"
                  "FORBIDDEN PHRASES:\n"
    "- 'Here is the article'\n"
    "- 'As a business'\n"
    "- 'In conclusion'\n"
    "- 'We recommend'\n\n"

    "- No markdown, bullets, or special characters\n"

    "\n No ###, no **, DO NOT USE THEM!"

    "- 'deals'\n\n"
                "BAD EXAMPLE TO AVOID:\n"
                "'Here's your article: ...'\n\n"
                "GOOD EXAMPLE FORMAT:\n"
                "'ABC plasma at 123 Main St offers... Their services include...'\n\n"
                "DEFECTIVE ARTICLE:\n"
                f"{article}\n\n"
                 "ARTICLE MUST BE FORMATTED IN <p style='line-height: 1.6;'>   </P> WITH INLINE CSS NOTHING EXTRA BEFORE OR AFTER <P style=''> </P>."
   "YOU WRITE FROM SELLPLASMANEAR PERSPECTIVE YOU DONT WORK FOR THIS CENTER! YOU WILL REMOVE ASTERISKS AND * AND HASTAGS #"
                "RETURN ONLY THE FIXED ARTICLE, NO EXTRA REMARKS OR COMMENTS, DO NOT SAY 'HERE IS YOUR ARTICLE', GIVE ONLY THE ARTICLE!"
            )
            try:
                response = ollama.chat(
                    model='deepseek-r1:14b',
                    messages=[{'role': 'user', 'content': prompt}]
                )
                return self._clean_output(response['message']['content'].strip())
            except Exception as e:
                print(f"Fixer error: {e}")
                return article



    def _clean_output(self, text):
        """Clean model output."""
        return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

########################################################################
# Helper Functions
########################################################################
def clean_filename(name):
    """Clean a string to be a valid filename."""
    return re.sub(r'[\\/*?:"<>|]', "", name).strip()

async def save_checkpoint(index):
    """Save current shop index to a checkpoint file."""
    with open(CHECKPOINT_FILE, "w") as f:
        f.write(str(index))

async def load_checkpoint():
    """Load the current shop index from a checkpoint file."""
    try:
        with open(CHECKPOINT_FILE, "r") as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return 0

def validate_shop_data(shop_data):
    """Validate required fields in shop data and content length."""
    required_metadata = ['name', 'city', 'state']
    meta = shop_data.get('metadata', {})

    missing = [field for field in required_metadata if not meta.get(field)]
    if missing:
        return False, f"Missing metadata fields: {missing}"

    return True, "Valid shop data"
def replace_unicode_dashes(s):
    """
    Replace en dash or em dash with a normal ASCII hyphen.
    """
    if not isinstance(s, str):
        return s
    return s.replace("–", "-").replace("—", "-")

########################################################################
# Processing Functions
########################################################################
import math

def split_text_evenly_dynamic(text, max_chunk_size):
    """
    Splits the text into an equal number of chunks, where each chunk is
    as close as possible to the same length and no chunk exceeds max_chunk_size.
    """
    total_length = len(text)
    # Determine the number of chunks needed such that each chunk is at most max_chunk_size.
    n_chunks = math.ceil(total_length / max_chunk_size)
    # Determine the equal chunk size.
    chunk_size = math.ceil(total_length / n_chunks)

    chunks = []
    for i in range(n_chunks):
        start = i * chunk_size
        end = start + chunk_size
        chunks.append(text[start:end])
    return chunks

async def process_shop(shop_path, ai_client,  total):
    """Process a single shop JSON file."""


    try:
        with open(shop_path, 'r', encoding='utf-8', errors='replace') as f:
            shop_data = json.load(f)
    except Exception as e:
        print(f"Error loading {shop_path}: {e}")
        return

    if await should_skip_shop(shop_data):
        print("d")
        print(f"Skipping giant brand: {shop_data.get('metadata', {}).get('name')}")
        return
    is_valid, validation_msg = validate_shop_data(shop_data)
    if not is_valid:
        print(f"Skipping {shop_path}: {validation_msg}")
        return

    meta = shop_data['metadata']
    lat = meta['latitude']
    content = shop_data['content']
    visual_text = ' '.join([v.get('extracted_text', '')
                          for v in shop_data.get('visual_analysis', [])])
    bing_text = ' '.join([v.get('content', '')
                          for v in shop_data.get('bing_results', [])])
    bing_text2 = ' '.join([v.get('url', '')
                          for v in shop_data.get('bing_results', [])])
    wiki = shop_data['wikipedia_data']
    if not wiki:
      wiki = ''
    output_path = Path(BASE_OUTPUT_DIR) / meta['state'].lower() / meta['city'].lower()
    output_path.mkdir(parents=True, exist_ok=True)
    filename = output_path / f"{clean_filename(meta['latitude'])}.txt"
    if filename.exists():
        print(f"Skipping {shop_path}: Generated article already exists at {filename}")
        return

    combined_text = (
        f"Business Name: {meta['name']}\n"
        f"Location: {meta['streetAddress']}\n"
        f"Services: {content}\n"
        f"Additional Details: {str(visual_text) + str(bing_text) + str(bing_text2)+str(wiki)}"
    )

    try:
        summary = await chunked_summarize(ai_client.qwen, combined_text)
        print(f"[AI] Final summary: {summary}")
    except Exception as e:
        print(f"Summarization failed: {e}")
        return




    article_prompt = (
    "ROLE: Professional writer for SellPlasmaNear.com  writing about plasma donation center's services\n"
    "TASK: Create article about this shop using ONLY the provided data\n\n"
    "WE WANT AN ARTICLE ON SERVICES BASED ON INFO, NOT SHIT"
    "KEEP SEO IN MIND FOR MOST LIKELY SEARCHED TERMS FOR ARTICLE FOR SellPlasmaNear.COM\n"
    "INCLUDE TERMS LIKE 'NEAR ME' 'Sell' ECT FOR SellPlasmaNear.COM AND YOU WRITE IN A STYLE LIKE YOU ARE REPORTING ON THEIR SERVICES PERSPECTIVE AS A SellPlasmaNear.COM WRITER, YOU WORK FOR SellPlasmaNear.COM WRITING ABOUT WHAT Plasma Donation Centers HAVE TO OFFER OFF GIVEN WEBSITE INFO WE GATHERED ON THEM FOR THE MAIN PAGE OF SellPlasmaNear.COM!\n"
    "STRICT RULES:\n"
    "EXPLAIN PLASMA DONATION "

    "article must be long and in detail if able"
                "MAKE SURE IT IS FRIENDY CONSUMER LANGUAGE READABLE ARTICLE ABOUT THE SPECIFIC SHOP SERVICES"
    "1. BEGIN IMMEDIATELY WITH ARTICLE CONTENT. "

     "You must only provide the article wanted and no other extra text, no *. NO EXTRA COMMENTS BEFORE OR AFTER THE ARTICLE, JUST RESPOND WITH ONLY THE ARTICLE."
    "2. NO text before/after the article\n"
    "3. NO markdown, asterisks, or special formatting\n"
    "4. INCLUDE NO URLS\n\n"
    "You write about the shop not the webpage."
    "REPORT ON THE DONATION CENTER AND PROCEDURES HISTORY GENERALLY as a sellplasmanear.com writer"
    "CONTENT STRUCTURE:\n"
    "DO NOT REPORT ON COOKIES, POLICIES, OR SCRIPTS"
    "IF NOT ENOUGH INFO JUST SAY 'NOT ENOUGH INFO'"
      "ENSURE THAT IT IS A PROPER ARTICLE BASED ON A PLASMA DONATION CENTER"
    "Long PARAGRAPHS ON SERVICES IF NOT SERVICES LOCATION ECT, NOT SPECIFIC DEALS OR HOLIDAY STUFF"
  "Do not say 'explore our' YOU WORK FOR SellPlasmaNear.COM, they arent our things its theirs and we are reporting"
    "FORBIDDEN PHRASES:\n"
    "- 'Here is the article'\n"
    "- 'As a business'\n"
    "- 'In conclusion'\n"
    "- 'We recommend'\n\n"
       "- 'Deals"

    "- 'explore our"
    "- 'deal saturday'\n\n"


    "BAD EXAMPLE (REJECT THIS):\n"
    "'Here's your article about Example Shop: They provide...'\n\n"
    "'Here's your article about...' [WRONG START, ONLY PROVIDE THE ARTICLE]\n"


    "GOOD EXAMPLE (COPY THIS FORMAT):\n"
    "'453 Plasma Center in downtown Miami offers... Their team specializes in... Make Money ...'\n"
    "'123 Plasma Center offers... Their services include... They Began in ...'\n\n"

    "BUSINESS DATA:\n"
    f"Name: {meta['name']}\n"
    f"Address: {meta['streetAddress']}\n"
    f"Services: {summary}\n"


    "- Natural language with location keywords\n"
    "- Address reader as 'you'\n\n"
    "- No markdown, or WEIRD special characters\n"
    "NO # or * OR WEIRD CHARACTERS! "
    "\n No ###, no **, DO NOT USE THEM!"

    "ARTICLE MUST BE AS LONG AS POSSIBLE ADD FAQ ECT"

    "EXPLAIN POTENTAL MONEY AND ALSO HOW YOU SHOULD PROCEED TO THEM AND STEP BY STEP DIRECTIONS ECT. "
    "ARTICLE MUST BE FORMATTED IN <p style='line-height: 1.6;'>   </P> WITH INLINE CSS NOTHING EXTRA BEFORE OR AFTER <P style=''> </P>."
    " STYLE THE CSS YOURSELF, MAKE THE ARTICLE AS LONG AS POSSIBLE, BULLET POINTS ECT, ONLY IN THE <p></p> AND MAKE IT NICE!"
    "SIMPLE INLINE CSS BUT ENSURE IT WILL RENDER NICELY AND NOT BUNCHED UP TEXT"
    " DO NOT USE # or *! "
       "YOU WRITE FROM SELLPLASMANEAR PERSPECTIVE YOU DONT WORK FOR THIS CENTER! YOU WILL REMOVE ASTERISKS AND * AND HASTAGS # "
    "IMPERATIVE: RESPOND ONLY WITH THE RAW ARTICLE TEXT - NO OTHER CONTENT OR REMARKS!"
)
    ()
    try:
        article = await ai_client.deepseek.generate_article(article_prompt)
        print(f"[AI] Generated article: {article}")
    except Exception as e:
        print(f"Article generation failed: {e}")
        return

    is_valid, validation_msg = await ai_client.deepseek.validate_article(article)
    if not is_valid:
        print("Article failed validation. Attempting fix...")

        article = await ai_client.deepseek.fixer(article, validation_msg)


        if not await ai_client.deepseek.validate_article(article):
            print("final validation failed")
            return

    if article:
        print("ARTICLE VALID")
        print(article)
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(article)
        if lat:
            print("HERE WITH SLUG")
            try:
                lat_float = float(replace_unicode_dashes(lat))

                lat_rounded = truncate_to_decimals(lat_float, 3)
            except ValueError:
                lat_rounded = replace_unicode_dashes(lat)
            print(lat_rounded)
            payload = {
                "latitude": lat_rounded,
                "description": article
            }
            endpoint = "https://www.sellplasmanear.com/e/s/s/update/"

            async with httpx.AsyncClient() as client:

                response = await client.post(endpoint, json=payload, timeout = 50)
                print('s', response)




########################################################################
# Main Execution
########################################################################
class AIClient:
    def __init__(self):
        self.deepseek = DeepSeekClient()
        self.qwen = QwenClient()
# Define the list of giant brands to skip
GIANT_BRANDS = [


]

def truncate_to_decimals(number, decimals=5):
    factor = 10 ** decimals
    return math.floor(number * factor) / factor

async def should_skip_shop(shop_data):
    print("ddd")
    meta = shop_data.get('metadata', {})

    lat = meta.get('latitude')  # Use `.get()` to avoid KeyError
    if not lat:  # If address is missing or empty, return False (do not skip)
        return FALSE
    try:
          lat_float = float(replace_unicode_dashes(lat))

          lat_rounded = truncate_to_decimals(lat_float, 3)

    except ValueError:
            lat_rounded = replace_unicode_dashes(lat)
            print(lat_rounded)
    print(lat_rounded)
    payload = {
        "latitude": lat_rounded,
        "description": ""
    }
    endpoint = "https://www.sellplasmanear.com/e/s/s/update/"

    async with httpx.AsyncClient() as client:
        try:
            response = await client.post(endpoint, json=payload, timeout=50)
            print(response.status_code)  # Debugging

            if response.status_code == 300:
                return True  # Skip processing this shop

        except httpx.RequestError as e:
            print(f"Request error: {e}")  # Log request failures

    return False  # Default: do not skip
async def main():


    shop_files = list(Path(BASE_INPUT_DIR).rglob('*.json'))
    total_shops = len(shop_files)
    ai_client = AIClient()
    for i, shop_path in enumerate(shop_files):
        await process_shop(shop_path, ai_client, total_shops)
        if SHOPDD:
          break

    print("Processing completed successfully")

if __name__ == "__main__":
    await main()

nohup: appending output to 'nohup.out'
ddd
41.774
300
d
Skipping giant brand: CSL Plasma Manchester, CT
ddd
33.348
300
d
Skipping giant brand: BioLife Plasma Services Tempe, AZ
ddd
40.271
300
d
Skipping giant brand: BioLife Plasma Services Orem, UT
ddd
40.274
300
d
Skipping giant brand: Grifols Biomat USA Orem, UT
ddd
37.657
300
d
Skipping giant brand: Grifols Cedar City, UT
ddd
44.169
300
d
Skipping giant brand: BioLife Plasma Services Mankato, MN
ddd
35.981
300
d
Skipping giant brand: Parachute Plasma Donation Center Crossville, TN
ddd
26.206
300
d
Skipping giant brand: Octapharma Plasma North Lauderdale, FL
ddd
43.136
300
d
Skipping giant brand: Grifols Talecris Plasma Resources Milwaukee, WI
ddd
43.043
300
d
Skipping giant brand: Grifols PlasmaCare Donation Center Milwaukee, WI
ddd
43.178
300
d
Skipping giant brand: Octapharma Plasma Milwaukee, WI
ddd
43.12
200
Split into 5 chunks ~5000 chars each.
starting 1
[AI] QwenClient.fast_summarize response: - Location: 10210 W Silver Sprin

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
